In [20]:
import os 
from dotenv import load_dotenv 

load_dotenv()

MODEL = "llama2"


In [21]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

model = Ollama(model=MODEL)
model.invoke("Hello, how are you?")


C:\Users\Aadil\AppData\Local\Temp\ipykernel_18500\1684374434.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings()


" Hello! I'm just an AI, I don't have feelings or emotions like humans do, so I can't really experience the world in the same way that you do. However, I'm here to help answer any questions you may have or provide assistance with any tasks you might need help with. Is there anything specific you would like to know or discuss?"

In [22]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Hello, how are you?")

# not applciable for this usecase as it was already done in the model

'\nI\'m just an AI, I don\'t have feelings or emotions like humans do, so I can\'t really experience things like "hello" or "how are you?" in the way that a living being would. However, I\'m here to help answer any questions you may have or assist you in any way I can. Is there something specific you\'d like to know or discuss?'

In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('Profile.pdf')
pages = loader.load_and_split()
pages


[Document(metadata={'producer': 'Apache FOP Version 2.2', 'creator': 'PyPDF', 'creationdate': '2025-07-14T14:25:13+00:00', 'title': 'Resume', 'author': 'LinkedIn', 'subject': 'Resume generated from profile', 'source': 'Profile.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="Contact\nmohammadghani2003@gmail.com\nwww.linkedin.com/in/muhmmad-\naadil-ghani (LinkedIn)\naadil29.github.io/ProjectPortfolio/\n(Portfolio)\nTop Skills\nCloud Computing\nScalable Cloud Architecture\nReal-Time Data Processing\nCertifications\nCritical Thinking and Problem\nSolving\nCybersecurity Foundations\nJPMorgan Chase & Co\nCybersecurity Job Simulation\nCybersecurity Awareness:\nCybersecurity Terminology\nMuhammad Aadil Ghani\nCyber Security MSci | AI/ML & Cloud Developer\nUnited Kingdom\nSummary\nWith a Master’s in Computer Science with Cyber Security, I bring\nhands-on experience across both cyber security and AI/ML domains.\nMy work integrates deep learning with large-scale data and clou

In [24]:
#Prompt template that the model will use to answer the question about the document

from langchain_core.prompts import PromptTemplate
template = """
You are a helpful assistant. You will be provided with a document and a question about that document.
If the document contains the answer to the question, provide the answer. If not, say "I don't know".  

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="here is some context", question="Here is the answer?")



'\nYou are a helpful assistant. You will be provided with a document and a question about that document.\nIf the document contains the answer to the question, provide the answer. If not, say "I don\'t know".  \n\nContext: here is some context\n\nQuestion: Here is the answer?\n\n'

In [25]:
chain= prompt | model | parser

In [26]:
chain.invoke(
    {
    "context": "my name is ghani",
    "question": "what is my name?"
    }
)

" Hello Ghani! I don't know your name as it is not provided in the document you provided. Could you please provide more context or clarify which document you are referring to?"

In [27]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vecttorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding= embeddings
)

c:\Users\Aadil\Desktop\Local_RAG\.ragvenv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [29]:
retriever = vecttorstore.as_retriever()
retriever.invoke("education")

[Document(metadata={'producer': 'Apache FOP Version 2.2', 'creator': 'PyPDF', 'creationdate': '2025-07-14T14:25:13+00:00', 'title': 'Resume', 'author': 'LinkedIn', 'subject': 'Resume generated from profile', 'source': 'Profile.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}, page_content="Worked closely with cross-functional teams to understand data needs and\ndeliver actionable insights, enhancing decision-making at multiple levels within\nthe organisation.\nEducation\nCity, University of London\nIntegrated Master's degree,\xa0Computer Science with cyber security\nEastbury Community School\nMathematics, Computer Science and Chemistry\n\xa0 Page 3 of 3"),
 Document(metadata={'producer': 'Apache FOP Version 2.2', 'creator': 'PyPDF', 'creationdate': '2025-07-14T14:25:13+00:00', 'title': 'Resume', 'author': 'LinkedIn', 'subject': 'Resume generated from profile', 'source': 'Profile.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}, page_content="-Completed the STACK survey to ass

In [ ]:
from operator import itemgetter

chain = (
{
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")

}
| prompt | model | parser
)

chain.invoke(
    { "question": "What are my top 3 skills?" },
    )

' Based on the document provided, your top 3 skills are:\n\n1. Cloud Computing\n2. Scalable Cloud Architecture\n3. Real-Time Data Processing'

In [36]:
questions = [
    "What are my top 3 skills?",    
    "What is my education background?",
    "What is my work experience?",  
    "What links do I have included in my profile?"
]

for question in questions:
    print(f"Question: {question}")
    answer = chain.invoke({"question": question})
    print(f"Answer: {answer}")
    print()


Question: What are my top 3 skills?
Answer:  Based on the provided resume, your top 3 skills are:

1. Cloud Computing
2. Real-Time Data Processing
3. Cybersecurity Foundations

Question: What is my education background?
Answer: Based on the document provided, your education background is as follows:

* Master's degree in Computer Science with Cyber Security from City, University of London
* Mathematics, Computer Science and Chemistry at Eastbury Community School.

Question: What is my work experience?
Answer:  Based on the document provided, your work experience includes:

* TJX Europe: Sales Associate (October 2023 - January 2024)
	+ Supervised daily sales transactions at the till with precision and efficiency
	+ Organised and placed over 250 items per day to maintain high standards of store orderliness and visual appeal
	+ Delivered exceptional customer service to over 100 customers daily, swiftly adapting to fast-paced retail demands and maintaining a welcoming shopping environment
